# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

In [20]:
from IPython.display import Markdown, display


phase_1_text_section_1 = """
### CRISP-DM Phase 1: Business Understanding

The objective of this project is to help a used car dealership understand what factors most strongly influence a vehicle’s resale price.
By developing predictive models, the dealership can estimate prices more accurately and identify high-value vehicles for acquisition and sale.
The analysis supports better pricing, purchasing, and inventory management decisions, enabling data-driven operations.
Ultimately, the insights will help the dealership focus on vehicles that maximize profitability and meet customer demand efficiently.
"""
display(Markdown(phase_1_text_section_1))


### CRISP-DM Phase 1: Business Understanding

The objective of this project is to help a used car dealership understand what factors most strongly influence a vehicle’s resale price.
By developing predictive models, the dealership can estimate prices more accurately and identify high-value vehicles for acquisition and sale.
The analysis supports better pricing, purchasing, and inventory management decisions, enabling data-driven operations.
Ultimately, the insights will help the dealership focus on vehicles that maximize profitability and meet customer demand efficiently.


In [21]:

phase_1_text_section_1 = """
### CRISP-DM Phase 1: Data Problem Definition
This business understanding/problem  can be translated into a supervised regression modeling problem. Using the provided dataset, which contains attributes such as year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, drive, size, type, paint_color, state, and the target variable price, the goal is to build and evaluate predictive models that estimate vehicle price based on these input features.
The analysis will involve data preprocessing (handling missing values, encoding categorical variables, scaling numeric features), feature engineering (e.g., polynomial transformations), and modeling using algorithms such as  Linear Regression, LASSO, Ridge, Logistic Regression, and ARMA.
By analyzing the model coefficients and feature importance, we aim to identify the key predictors of car prices and quantify their effect, ultimately revealing which factors are most valued by consumers in the used car market.
"""
display(Markdown(phase_1_text_section_1))


### CRISP-DM Phase 1: Data Problem Definition
This business understanding/problem  can be translated into a supervised regression modeling problem. Using the provided dataset, which contains attributes such as year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, drive, size, type, paint_color, state, and the target variable price, the goal is to build and evaluate predictive models that estimate vehicle price based on these input features.
The analysis will involve data preprocessing (handling missing values, encoding categorical variables, scaling numeric features), feature engineering (e.g., polynomial transformations), and modeling using algorithms such as  Linear Regression, LASSO, Ridge, Logistic Regression, and ARMA.
By analyzing the model coefficients and feature importance, we aim to identify the key predictors of car prices and quantify their effect, ultimately revealing which factors are most valued by consumers in the used car market.


In [22]:
# =============================================================================
# CRISP-DM FRAMEWORK IMPLEMENTATION
# USED CAR PRICE MODELING — REGRESSION, CLASSIFICATION & FORECASTING
# =============================================================================

# PHASE 0: LIBRARY IMPORTS & INITIALIZATION
# -----------------------------------------------------------------------------
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import (
    r2_score, mean_squared_error,
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve
)
from statsmodels.tsa.arima.model import ARIMA
import warnings, os
warnings.filterwarnings("ignore")
from IPython.display import Markdown, display

# Create folder for saving images
os.makedirs("images", exist_ok=True)

# Start total timer
global_start = time.time()
print("\n=== Script started ===")

# =============================================================================
# PHASE 1: BUSINESS UNDERSTANDING
# -----------------------------------------------------------------------------
business_understanding = """
### Goal
--------
To help a used car dealership understand what factors influence used car prices
and predict price levels or identify high-value vehicles.

### Business Objectives:
------------------------
1. Predict used car resale prices using regression models.
2. Classify vehicles as "high-value" or "low-value" for better inventory targeting.
3. Forecast future average price trends for business strategy and acquisition planning.
4. Generate actionable insights to guide pricing, procurement, and marketing decisions.
"""
display(Markdown(business_understanding))


=== Script started ===



### Goal
--------
To help a used car dealership understand what factors influence used car prices
and predict price levels or identify high-value vehicles.

### Business Objectives:
------------------------
1. Predict used car resale prices using regression models.
2. Classify vehicles as "high-value" or "low-value" for better inventory targeting.
3. Forecast future average price trends for business strategy and acquisition planning.
4. Generate actionable insights to guide pricing, procurement, and marketing decisions.


### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [23]:
# =============================================================================
# PHASE 2: DATA UNDERSTANDING
# -----------------------------------------------------------------------------
df = pd.read_csv("data/vehicles.csv")
print(f"Initial dataset shape: {df.shape}")

# Inspect structure and missingness
print("\nSample columns:", df.columns.tolist()[:10])
print(df.isna().mean().sort_values(ascending=False).head(10))

Initial dataset shape: (426880, 18)

Sample columns: ['id', 'region', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer']
size            0.717675
cylinders       0.416225
condition       0.407852
VIN             0.377254
drive           0.305863
paint_color     0.305011
type            0.217527
manufacturer    0.041337
title_status    0.019308
model           0.012362
dtype: float64


### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [24]:
# =============================================================================
# PHASE 3: DATA PREPARATION
# -----------------------------------------------------------------------------
# Drop irrelevant columns
df.drop(columns=["id", "VIN"], inplace=True, errors='ignore')

# Fill missing categorical values
for col in ["condition", "fuel", "transmission", "drive", "type", "paint_color"]:
    if col in df.columns:
        df[col] = df[col].fillna("unknown").str.lower()

# Filter unrealistic prices and drop missing key fields
df = df.dropna(subset=["price", "year", "odometer"])
df = df[(df["price"] > 500) & (df["price"] < 100000)]
print(f"Data shape after cleaning: {df.shape}")

# Reduce high-cardinality columns
for col in ["region", "manufacturer", "model"]:
    if col in df.columns:
        top_cats = df[col].value_counts().nlargest(20).index
        df[col] = df[col].apply(lambda x: x if x in top_cats else "other")

# =============================================================================
# PHASE 3.1: EXPLORATORY DATA ANALYSIS (EDA)
# -----------------------------------------------------------------------------
sns.set(style="whitegrid")

# 1. Price distribution
plt.figure(figsize=(10,5))
sns.histplot(df["price"], bins=50, kde=True, color="skyblue")
plt.title("Distribution of Used Car Prices")
plt.tight_layout()
plt.savefig("images/Distribution of Used Car Prices.png")
plt.close()

# 2. Price by condition
plt.figure(figsize=(12,5))
sns.boxplot(x="condition", y="price", data=df)
plt.title("Price by Vehicle Condition")
plt.tight_layout()
plt.savefig("images/Price by Vehicle Condition.png")
plt.close()

# 3. Price by fuel type
plt.figure(figsize=(12,5))
sns.boxplot(x="fuel", y="price", data=df)
plt.title("Price by Fuel Type")
plt.tight_layout()
plt.savefig("images/Price by Fuel Type.png")
plt.close()

# 4. Price vs year
plt.figure(figsize=(12,5))
sns.scatterplot(x="year", y="price", data=df, alpha=0.5)
plt.title("Price vs Model Year")
plt.tight_layout()
plt.savefig("images/Price vs Model Year.png")
plt.close()

# --- Feature selection ---
target = "price"
X = df.drop(columns=[target])
y = df[target]

cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(exclude="object").columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Preprocessing pipelines ---
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

Data shape after cleaning: (379846, 16)


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [25]:
# --- Model Training Helper ---
results, model_timings = {}, {}

def fit_and_eval(name, estimator, param_grid=None, subsample=None):
    """Fit model, time it, and store metrics."""
    start = time.time()
    pipe = Pipeline([("preprocessor", preprocessor), ("model", estimator)])
    
    if param_grid:
        X_sub = X_train.sample(subsample, random_state=42) if subsample else X_train
        y_sub = y_train.loc[X_sub.index] if subsample else y_train
        grid = GridSearchCV(pipe, param_grid, cv=2, n_jobs=2, scoring="r2")
        grid.fit(X_sub, y_sub)
        best_model = grid.best_estimator_
        print(f"\n{name} best params: {grid.best_params_}")
    else:
        best_model = pipe.fit(X_train, y_train)
    
    duration = time.time() - start
    y_pred = best_model.predict(X_test)
    r2, rmse = r2_score(y_test, y_pred), np.sqrt(mean_squared_error(y_test, y_pred))
    
    results[name] = {"type": "regression", "R2": r2, "RMSE": rmse, "Duration": duration}
    model_timings[name] = duration
    print(f"{name} — R²: {r2:.3f}, RMSE: {rmse:.2f}, Duration: {duration:.2f}s")
    # Optional: coefficient interpretation
    if hasattr(best_model.named_steps['model'], 'coef_'):
        coefs = best_model.named_steps['model'].coef_
        plt.figure(figsize=(10, 5))
        sns.histplot(coefs, bins=30, kde=True)
        plt.title(f"Coefficient Distribution for {name}")
        #plt.show()
        plt.tight_layout()
        plt.savefig(f"images/Coefficient Distribution for {name}.png")
        plt.close()
    return best_model

# --- Train models ---
lin_model   = fit_and_eval("Linear Regression", LinearRegression())
ridge_model = fit_and_eval("Ridge Regression", Ridge(max_iter=5000), {"model__alpha":[0.1,1,10]})
lasso_model = fit_and_eval("Lasso Regression", Lasso(), {"model__alpha":[0.001,0.01,0.1]}, subsample=20000)

# =============================================================================
# LOGISTIC REGRESSION (Classification)
# -----------------------------------------------------------------------------
start = time.time()
median_price = df["price"].median()
df["high_price"] = (df["price"] > median_price).astype(int)

X = df.drop(columns=["price", "high_price"])
y = df["high_price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

num_feats = X.select_dtypes(exclude=["object"]).columns
cat_feats = X.select_dtypes(include=["object"]).columns

preprocessor_log = ColumnTransformer([
    ("num", StandardScaler(), num_feats),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_feats)
])

log_reg = LogisticRegression(penalty="l2", solver="liblinear", max_iter=1000)
pipe_log = Pipeline([("preprocessor", preprocessor_log), ("model", log_reg)])
pipe_log.fit(X_train, y_train)

y_pred = pipe_log.predict(X_test)
y_prob = pipe_log.predict_proba(X_test)[:,1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

duration = time.time() - start
results["Logistic Regression"] = {
    "type":"classification",
    "Accuracy":acc,"Precision":prec,"Recall":rec,
    "F1":f1,"ROC-AUC":roc_auc,"Duration":duration
}
model_timings["Logistic Regression"] = duration

print(f"Logistic Regression — Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}, ROC-AUC: {roc_auc:.3f}")

# ROC Plot
plt.figure(figsize=(6,4))
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label=f"AUC={roc_auc:.2f}", color="teal")
plt.plot([0,1],[0,1],'--',color='gray')
plt.title("ROC Curve — Logistic Regression")
plt.legend()
plt.tight_layout()
plt.savefig("images/ROC_LogisticRegression.png")
plt.close()

Linear Regression — R²: 0.470, RMSE: 10381.69, Duration: 5.28s

Ridge Regression best params: {'model__alpha': 10}
Ridge Regression — R²: 0.470, RMSE: 10381.53, Duration: 18.59s

Lasso Regression best params: {'model__alpha': 0.1}
Lasso Regression — R²: 0.465, RMSE: 10431.12, Duration: 20.64s
Logistic Regression — Accuracy: 0.863, Precision: 0.867, Recall: 0.856, F1: 0.861, ROC-AUC: 0.931


### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [26]:
# =============================================================================
# PHASE 5: EVALUATION (ARIMA Forecast + Model Comparison)
# -----------------------------------------------------------------------------
start_time = time.time()
try:
    df_year = df.copy()
    df_year["year"] = pd.to_numeric(df_year["year"], errors="coerce")
    df_year = df_year.dropna(subset=["year"])
    df_year = df_year[df_year["year"] > 1900]
    df_year["year"] = df_year["year"].astype(int)

    price_by_year = df_year.groupby("year")["price"].mean().sort_index()
    
    if price_by_year.shape[0] < 5:
        raise ValueError("Insufficient yearly data for ARIMA.")

    model = ARIMA(price_by_year, order=(1,0,1))
    fitted = model.fit()
    forecast_steps = 3
    forecast_result = fitted.get_forecast(steps=forecast_steps)
    forecast = forecast_result.predicted_mean
    conf_int = forecast_result.conf_int()

    actual_tail = price_by_year.tail(forecast_steps).values
    forecast_trimmed = forecast[:len(actual_tail)]
    rmse_arima = np.sqrt(mean_squared_error(actual_tail, forecast_trimmed))

    duration = time.time() - start_time
    model_timings["ARIMA"] = duration
    results["ARIMA"] = {"type": "regression", "R2": np.nan, "RMSE": rmse_arima, "Duration": duration}

    print(f"\nARIMA(1,0,1) RMSE: {rmse_arima:.2f}, Duration: {duration:.2f}s")

    plt.figure(figsize=(10,5))
    sns.lineplot(x=price_by_year.index, y=price_by_year.values, marker='o', label='Historical Avg Price')
    future_years = np.arange(price_by_year.index.max()+1, price_by_year.index.max()+1+forecast_steps)
    sns.lineplot(x=future_years, y=forecast, marker='x', label='Forecast', color='darkred')
    plt.fill_between(future_years, conf_int.iloc[:,0], conf_int.iloc[:,1], alpha=0.3, label="95% CI",color='lightgray',)
    plt.title("Average Used Car Price Trend & ARIMA Forecast")
    plt.xlabel("Year")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"images/Average Used Car Price Trend & ARIMA Forecast.png")
    plt.close()
    #plt.show()

except Exception as e:
    duration = time.time() - start_time
    model_timings["ARIMA"] = duration
    print(f"ARIMA model failed: {e}")



# --- Model evaluation summary ---
print("\n=== MODEL EVALUATION SUMMARY ===")
for k,v in results.items():
    if v["type"]=="regression" and not np.isnan(v.get("R2",np.nan)):
        print(f"{k}: R²={v['R2']:.3f}, RMSE={v['RMSE']:.2f}")
    elif v["type"]=="classification":
        print(f"{k}: Accuracy={v['Accuracy']:.3f}, Precision={v['Precision']:.3f}, Recall={v['Recall']:.3f}, F1={v['F1']:.3f}, ROC-AUC={v['ROC-AUC']:.3f}")
    else:
        print(f"{k}: RMSE={v['RMSE']:.2f}")

# Determine best model dynamically
reg_models = {k:v for k,v in results.items() if v["type"]=="regression"}
cls_models = {k:v for k,v in results.items() if v["type"]=="classification"}
best_reg = max(reg_models.items(), key=lambda x: x[1].get("R2",-np.inf)) if reg_models else None
best_cls = max(cls_models.items(), key=lambda x: x[1]["ROC-AUC"]) if cls_models else None
best_model_name, best_metrics = (best_cls if best_cls else best_reg)

print(f"\nBest Performing Model: {best_model_name}")

print("\n=== MODEL TRAINING DURATION SUMMARY ===")
for k,v in model_timings.items():
    print(f"{k:<20}: {v/60:.2f} min ({v:.1f} sec)")
total_time = time.time() - global_start
print(f"\n=== TOTAL SCRIPT RUNTIME: {total_time/60:.2f} min ({total_time:.1f} sec) ===")



ARIMA(1,0,1) RMSE: 12788.34, Duration: 0.65s

=== MODEL EVALUATION SUMMARY ===
Linear Regression: R²=0.470, RMSE=10381.69
Ridge Regression: R²=0.470, RMSE=10381.53
Lasso Regression: R²=0.465, RMSE=10431.12
Logistic Regression: Accuracy=0.863, Precision=0.867, Recall=0.856, F1=0.861, ROC-AUC=0.931
ARIMA: RMSE=12788.34

Best Performing Model: Logistic Regression

=== MODEL TRAINING DURATION SUMMARY ===
Linear Regression   : 0.09 min (5.3 sec)
Ridge Regression    : 0.31 min (18.6 sec)
Lasso Regression    : 0.34 min (20.6 sec)
Logistic Regression : 0.16 min (9.8 sec)
ARIMA               : 0.01 min (0.7 sec)

=== TOTAL SCRIPT RUNTIME: 2.96 min (177.4 sec) ===


### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.

In [27]:
# =============================================================================
# PHASE 6: DEPLOYMENT (Business Insights & Model Justification)
# -----------------------------------------------------------------------------
print("\n=== BUSINESS INSIGHTS & RECOMMENDATIONS ===")
if best_metrics["type"] == "regression":
    print(f"Best Model: {best_model_name} — R²={best_metrics['R2']:.3f}, RMSE={best_metrics['RMSE']:.2f}")
    print("""
• Newer model years and lower odometer readings strongly increase resale value.
• Good vehicle condition and reputable manufacturers increase price.
• High mileage and poor condition decrease value.
• Ridge/Lasso provide robust continuous price predictions.
• Dealership should prioritize newer, low-mileage, clean vehicles.
""")
else:
    print(f"Best Model: {best_model_name} — Accuracy={best_metrics['Accuracy']:.3f}, ROC-AUC={best_metrics['ROC-AUC']:.3f}")
    print("""
• The classification model effectively identifies high-value vehicles.
• Use predicted probability (ROC-AUC) to segment vehicles for premium listings.
• Focus on features driving higher resale probability (year, odometer, condition).
""")
# === RATIONALE: Why Logistic Regression is Best ===
print("\n=== RATIONALE: BEST MODEL SELECTION ===")
print("""
Although regression models predict exact price values (R² ~0.47), Logistic Regression classifies vehicles 
as high vs low price and achieves very high accuracy (86%) and ROC-AUC (0.93). 
In practical business applications, identifying premium vehicles reliably is more actionable than small 
differences in continuous price prediction. Therefore, Logistic Regression is selected as the best model 
for high-value vehicle identification, segmenting inventory, and pricing decisions.
""")


=== BUSINESS INSIGHTS & RECOMMENDATIONS ===
Best Model: Logistic Regression — Accuracy=0.863, ROC-AUC=0.931

• The classification model effectively identifies high-value vehicles.
• Use predicted probability (ROC-AUC) to segment vehicles for premium listings.
• Focus on features driving higher resale probability (year, odometer, condition).


=== RATIONALE: BEST MODEL SELECTION ===

Although regression models predict exact price values (R² ~0.47), Logistic Regression classifies vehicles 
as high vs low price and achieves very high accuracy (86%) and ROC-AUC (0.93). 
In practical business applications, identifying premium vehicles reliably is more actionable than small 
differences in continuous price prediction. Therefore, Logistic Regression is selected as the best model 
for high-value vehicle identification, segmenting inventory, and pricing decisions.



In [31]:
code_base = """
### Providing complete code base for reference 
"""
display(Markdown(code_base))


### Providing complete code base for reference 


In [ ]:
# =============================================================================
# CRISP-DM FRAMEWORK IMPLEMENTATION
# USED CAR PRICE MODELING — REGRESSION, CLASSIFICATION & FORECASTING
# =============================================================================

# PHASE 0: LIBRARY IMPORTS & INITIALIZATION
# -----------------------------------------------------------------------------
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import (
    r2_score, mean_squared_error,
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve
)
from statsmodels.tsa.arima.model import ARIMA
import warnings, os
warnings.filterwarnings("ignore")

# Create folder for saving images
os.makedirs("images", exist_ok=True)

# Start total timer
global_start = time.time()
print("\n=== Script started ===")

# =============================================================================
# PHASE 1: BUSINESS UNDERSTANDING
# -----------------------------------------------------------------------------
"""
Goal:
------
To help a used car dealership understand what factors influence used car prices
and predict price levels or identify high-value vehicles.

Business Objectives:
--------------------
1. Predict used car resale prices using regression models.
2. Classify vehicles as "high-value" or "low-value" for better inventory targeting.
3. Forecast future average price trends for business strategy and acquisition planning.
4. Generate actionable insights to guide pricing, procurement, and marketing decisions.
"""
# =============================================================================
# PHASE 2: DATA UNDERSTANDING
# -----------------------------------------------------------------------------
df = pd.read_csv("data/vehicles.csv")
print(f"Initial dataset shape: {df.shape}")

# Inspect structure and missingness
print("\nSample columns:", df.columns.tolist()[:10])
print(df.isna().mean().sort_values(ascending=False).head(10))

# =============================================================================
# PHASE 3: DATA PREPARATION
# -----------------------------------------------------------------------------
# Drop irrelevant columns
df.drop(columns=["id", "VIN"], inplace=True, errors='ignore')

# Fill missing categorical values
for col in ["condition", "fuel", "transmission", "drive", "type", "paint_color"]:
    if col in df.columns:
        df[col] = df[col].fillna("unknown").str.lower()

# Filter unrealistic prices and drop missing key fields
df = df.dropna(subset=["price", "year", "odometer"])
df = df[(df["price"] > 500) & (df["price"] < 100000)]
print(f"Data shape after cleaning: {df.shape}")

# Reduce high-cardinality columns
for col in ["region", "manufacturer", "model"]:
    if col in df.columns:
        top_cats = df[col].value_counts().nlargest(20).index
        df[col] = df[col].apply(lambda x: x if x in top_cats else "other")

# =============================================================================
# PHASE 3.1: EXPLORATORY DATA ANALYSIS (EDA)
# -----------------------------------------------------------------------------
sns.set(style="whitegrid")

# 1. Price distribution
plt.figure(figsize=(10,5))
sns.histplot(df["price"], bins=50, kde=True, color="skyblue")
plt.title("Distribution of Used Car Prices")
plt.tight_layout()
plt.savefig("images/Distribution of Used Car Prices.png")
plt.close()

# 2. Price by condition
plt.figure(figsize=(12,5))
sns.boxplot(x="condition", y="price", data=df)
plt.title("Price by Vehicle Condition")
plt.tight_layout()
plt.savefig("images/Price by Vehicle Condition.png")
plt.close()

# 3. Price by fuel type
plt.figure(figsize=(12,5))
sns.boxplot(x="fuel", y="price", data=df)
plt.title("Price by Fuel Type")
plt.tight_layout()
plt.savefig("images/Price by Fuel Type.png")
plt.close()

# 4. Price vs year
plt.figure(figsize=(12,5))
sns.scatterplot(x="year", y="price", data=df, alpha=0.5)
plt.title("Price vs Model Year")
plt.tight_layout()
plt.savefig("images/Price vs Model Year.png")
plt.close()

# =============================================================================
# PHASE 4: MODELING
# -----------------------------------------------------------------------------
# --- Feature selection ---
target = "price"
X = df.drop(columns=[target])
y = df[target]

cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(exclude="object").columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Preprocessing pipelines ---
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

# --- Model Training Helper ---
results, model_timings = {}, {}

def fit_and_eval(name, estimator, param_grid=None, subsample=None):
    """Fit model, time it, and store metrics."""
    start = time.time()
    pipe = Pipeline([("preprocessor", preprocessor), ("model", estimator)])
    
    if param_grid:
        X_sub = X_train.sample(subsample, random_state=42) if subsample else X_train
        y_sub = y_train.loc[X_sub.index] if subsample else y_train
        grid = GridSearchCV(pipe, param_grid, cv=2, n_jobs=2, scoring="r2")
        grid.fit(X_sub, y_sub)
        best_model = grid.best_estimator_
        print(f"\n{name} best params: {grid.best_params_}")
    else:
        best_model = pipe.fit(X_train, y_train)
    
    duration = time.time() - start
    y_pred = best_model.predict(X_test)
    r2, rmse = r2_score(y_test, y_pred), np.sqrt(mean_squared_error(y_test, y_pred))
    
    results[name] = {"type": "regression", "R2": r2, "RMSE": rmse, "Duration": duration}
    model_timings[name] = duration
    print(f"{name} — R²: {r2:.3f}, RMSE: {rmse:.2f}, Duration: {duration:.2f}s")
    # Optional: coefficient interpretation
    if hasattr(best_model.named_steps['model'], 'coef_'):
        coefs = best_model.named_steps['model'].coef_
        plt.figure(figsize=(10, 5))
        sns.histplot(coefs, bins=30, kde=True)
        plt.title(f"Coefficient Distribution for {name}")
        #plt.show()
        plt.tight_layout()
        plt.savefig(f"images/Coefficient Distribution for {name}.png")
        plt.close()
    return best_model

# --- Train models ---
lin_model   = fit_and_eval("Linear Regression", LinearRegression())
ridge_model = fit_and_eval("Ridge Regression", Ridge(max_iter=5000), {"model__alpha":[0.1,1,10]})
lasso_model = fit_and_eval("Lasso Regression", Lasso(), {"model__alpha":[0.001,0.01,0.1]}, subsample=20000)

# =============================================================================
# LOGISTIC REGRESSION (Classification)
# -----------------------------------------------------------------------------
start = time.time()
median_price = df["price"].median()
df["high_price"] = (df["price"] > median_price).astype(int)

X = df.drop(columns=["price", "high_price"])
y = df["high_price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

num_feats = X.select_dtypes(exclude=["object"]).columns
cat_feats = X.select_dtypes(include=["object"]).columns

preprocessor_log = ColumnTransformer([
    ("num", StandardScaler(), num_feats),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_feats)
])

log_reg = LogisticRegression(penalty="l2", solver="liblinear", max_iter=1000)
pipe_log = Pipeline([("preprocessor", preprocessor_log), ("model", log_reg)])
pipe_log.fit(X_train, y_train)

y_pred = pipe_log.predict(X_test)
y_prob = pipe_log.predict_proba(X_test)[:,1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

duration = time.time() - start
results["Logistic Regression"] = {
    "type":"classification",
    "Accuracy":acc,"Precision":prec,"Recall":rec,
    "F1":f1,"ROC-AUC":roc_auc,"Duration":duration
}
model_timings["Logistic Regression"] = duration

print(f"Logistic Regression — Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}, ROC-AUC: {roc_auc:.3f}")

# ROC Plot
plt.figure(figsize=(6,4))
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label=f"AUC={roc_auc:.2f}", color="teal")
plt.plot([0,1],[0,1],'--',color='gray')
plt.title("ROC Curve — Logistic Regression")
plt.legend()
plt.tight_layout()
plt.savefig("images/ROC_LogisticRegression.png")
plt.close()

# =============================================================================
# PHASE 5: EVALUATION (ARIMA Forecast + Model Comparison)
# -----------------------------------------------------------------------------
start_time = time.time()
try:
    df_year = df.copy()
    df_year["year"] = pd.to_numeric(df_year["year"], errors="coerce")
    df_year = df_year.dropna(subset=["year"])
    df_year = df_year[df_year["year"] > 1900]
    df_year["year"] = df_year["year"].astype(int)

    price_by_year = df_year.groupby("year")["price"].mean().sort_index()
    
    if price_by_year.shape[0] < 5:
        raise ValueError("Insufficient yearly data for ARIMA.")

    model = ARIMA(price_by_year, order=(1,0,1))
    fitted = model.fit()
    forecast_steps = 3
    forecast_result = fitted.get_forecast(steps=forecast_steps)
    forecast = forecast_result.predicted_mean
    conf_int = forecast_result.conf_int()

    actual_tail = price_by_year.tail(forecast_steps).values
    forecast_trimmed = forecast[:len(actual_tail)]
    rmse_arima = np.sqrt(mean_squared_error(actual_tail, forecast_trimmed))

    duration = time.time() - start_time
    model_timings["ARIMA"] = duration
    results["ARIMA"] = {"type": "regression", "R2": np.nan, "RMSE": rmse_arima, "Duration": duration}

    print(f"\nARIMA(1,0,1) RMSE: {rmse_arima:.2f}, Duration: {duration:.2f}s")

    plt.figure(figsize=(10,5))
    sns.lineplot(x=price_by_year.index, y=price_by_year.values, marker='o', label='Historical Avg Price')
    future_years = np.arange(price_by_year.index.max()+1, price_by_year.index.max()+1+forecast_steps)
    sns.lineplot(x=future_years, y=forecast, marker='x', label='Forecast', color='darkred')
    plt.fill_between(future_years, conf_int.iloc[:,0], conf_int.iloc[:,1], alpha=0.3, label="95% CI",color='lightgray',)
    plt.title("Average Used Car Price Trend & ARIMA Forecast")
    plt.xlabel("Year")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"images/Average Used Car Price Trend & ARIMA Forecast.png")
    plt.close()
    #plt.show()

except Exception as e:
    duration = time.time() - start_time
    model_timings["ARIMA"] = duration
    print(f"ARIMA model failed: {e}")



# --- Model evaluation summary ---
print("\n=== MODEL EVALUATION SUMMARY ===")
for k,v in results.items():
    if v["type"]=="regression" and not np.isnan(v.get("R2",np.nan)):
        print(f"{k}: R²={v['R2']:.3f}, RMSE={v['RMSE']:.2f}")
    elif v["type"]=="classification":
        print(f"{k}: Accuracy={v['Accuracy']:.3f}, Precision={v['Precision']:.3f}, Recall={v['Recall']:.3f}, F1={v['F1']:.3f}, ROC-AUC={v['ROC-AUC']:.3f}")
    else:
        print(f"{k}: RMSE={v['RMSE']:.2f}")

# Determine best model dynamically
reg_models = {k:v for k,v in results.items() if v["type"]=="regression"}
cls_models = {k:v for k,v in results.items() if v["type"]=="classification"}
best_reg = max(reg_models.items(), key=lambda x: x[1].get("R2",-np.inf)) if reg_models else None
best_cls = max(cls_models.items(), key=lambda x: x[1]["ROC-AUC"]) if cls_models else None
best_model_name, best_metrics = (best_cls if best_cls else best_reg)

print(f"\nBest Performing Model: {best_model_name}")

print("\n=== MODEL TRAINING DURATION SUMMARY ===")
for k,v in model_timings.items():
    print(f"{k:<20}: {v/60:.2f} min ({v:.1f} sec)")
total_time = time.time() - global_start
print(f"\n=== TOTAL SCRIPT RUNTIME: {total_time/60:.2f} min ({total_time:.1f} sec) ===")

# =============================================================================
# PHASE 6: DEPLOYMENT (Business Insights & Model Justification)
# -----------------------------------------------------------------------------
print("\n=== BUSINESS INSIGHTS & RECOMMENDATIONS ===")
if best_metrics["type"] == "regression":
    print(f"Best Model: {best_model_name} — R²={best_metrics['R2']:.3f}, RMSE={best_metrics['RMSE']:.2f}")
    print("""
• Newer model years and lower odometer readings strongly increase resale value.
• Good vehicle condition and reputable manufacturers increase price.
• High mileage and poor condition decrease value.
• Ridge/Lasso provide robust continuous price predictions.
• Dealership should prioritize newer, low-mileage, clean vehicles.
""")
else:
    print(f"Best Model: {best_model_name} — Accuracy={best_metrics['Accuracy']:.3f}, ROC-AUC={best_metrics['ROC-AUC']:.3f}")
    print("""
• The classification model effectively identifies high-value vehicles.
• Use predicted probability (ROC-AUC) to segment vehicles for premium listings.
• Focus on features driving higher resale probability (year, odometer, condition).
""")
# === RATIONALE: Why Logistic Regression is Best ===
print("\n=== RATIONALE: BEST MODEL SELECTION ===")
print("""
Although regression models predict exact price values (R² ~0.47), Logistic Regression classifies vehicles 
as high vs low price and achieves very high accuracy (86%) and ROC-AUC (0.93). 
In practical business applications, identifying premium vehicles reliably is more actionable than small 
differences in continuous price prediction. Therefore, Logistic Regression is selected as the best model 
for high-value vehicle identification, segmenting inventory, and pricing decisions.
""")